## 1. Загрузите данные. Используйте датасет с ирисами. Его можно загрузить непосредственно из библиотеки Sklearn. В данных оставьте только 2 класса: Iris Versicolor, Iris Virginica.

Текст, выделенный полужирным шрифтом# Новый раздел

In [1]:
from sklearn import datasets
import numpy as np
import pandas

# загружаем датасет iris
ds = datasets.load_iris()

# конвертируем датасет в pandas, чтобы он выглядел как необработанный 🤷‍♂️
names = list(map(lambda t : [ds['target_names'][int(t)]],ds['target']))
data = pandas.DataFrame(data=np.hstack((ds['data'],names)),
                       columns=ds['feature_names'] + ['target'])

# не знаю почему, но после конвертации тип колонок в становится string, поэтому
# конвертируем колонки обратно во float 
for feature in ds['feature_names']:
  data[feature] = data[feature].astype(float)

# оставляем в датасете только "Iris Versicolor" and "Iris Virginica"
data = data[data['target'] != 'setosa'].reset_index(drop=True)

# проверяем датасет на отсутствие битых данных
# data['sepal length (cm)'].unique()
data = data.dropna()

# выводим информацию о данных
data.info()
data

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   sepal length (cm)  100 non-null    float64
 1   sepal width (cm)   100 non-null    float64
 2   petal length (cm)  100 non-null    float64
 3   petal width (cm)   100 non-null    float64
 4   target             100 non-null    object 
dtypes: float64(4), object(1)
memory usage: 4.7+ KB


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,7.0,3.2,4.7,1.4,versicolor
1,6.4,3.2,4.5,1.5,versicolor
2,6.9,3.1,4.9,1.5,versicolor
3,5.5,2.3,4.0,1.3,versicolor
4,6.5,2.8,4.6,1.5,versicolor
...,...,...,...,...,...
95,6.7,3.0,5.2,2.3,virginica
96,6.3,2.5,5.0,1.9,virginica
97,6.5,3.0,5.2,2.0,virginica
98,6.2,3.4,5.4,2.3,virginica


Подготовка данных перед тестированием различных реализаций градиентного спуска

In [2]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

features_x = ['sepal length (cm)','sepal width (cm)','petal length (cm)','petal width (cm)']
result_y = 'target'

BASE_X = data[features_x]

le = LabelEncoder()
le.fit(data[result_y])

BASE_Y = le.transform(data[result_y])

# 80%/20%
TRAIN_X, TEST_X, TRAIN_Y, TEST_Y = train_test_split(BASE_X, BASE_Y, test_size = 0.2, random_state = 0)
print(TRAIN_X)
print(TRAIN_Y)

    sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
43                5.0               2.3                3.3               1.0
62                6.8               3.0                5.5               2.1
3                 5.5               2.3                4.0               1.3
71                5.6               2.8                4.9               2.0
45                5.7               3.0                4.2               1.2
..                ...               ...                ...               ...
96                6.3               2.5                5.0               1.9
67                7.7               3.8                6.7               2.2
64                5.8               2.8                5.1               2.4
47                6.2               2.9                4.3               1.3
44                5.6               2.7                4.2               1.3

[80 rows x 4 columns]
[0 1 0 1 0 0 0 1 1 1 1 1 1 1 1 0 0 1 1 1 0 1 0 0 0 0 

## 2. Самостоятельно реализуйте логистическую регрессию, без использования метода LogisticRegression из библиотеки. Можете использовать библиотеки pandas, numpy, math для реализации. Оформите в виде функции. *Оформите в виде класса с методами.

In [3]:
# Собственная реализация логистической регрессии
import time

class MyLogisticRegression:
  def __init__(self,epoch=1000,learning_rate=0.01,learn_method='sgd',stop_rate=0.0001): 
    self.epoch = epoch # здесь храним количество итераций
    self.learn_method = learn_method # здесь храним тип метода обучения
    self.learning_rate = learning_rate # здесь храним шаг
    self.stop_rate = stop_rate # значение, при котором остановить обучение
    self.learn_func = None # ссылка на функцию обучения
    self.cost_func = None # ссылка на функцию потерь

    self.weights = None
    self.epoch_passed = 0
    self.learn_time = 0

    if(self.learn_method == 'sgd'):
      self.learn_func = self.learn_sgd
    elif(self.learn_method == 'rmsprop'):
      self.learn_func = self.learn_rmsprop
    elif(self.learn_method == 'adam'):
      self.learn_func = self.learn_adam
    elif(self.learn_method == 'nadam'):
      self.learn_func = self.learn_nadam
    else:
      raise Exception('Unknown learining method: {}'.format(self.learn_method))


  '''
    производит начальное инициализирование весов
  '''
  def init_weights(self,X):
    #return np.random.randn(X.shape[1], 1);
    return np.zeros((X.shape[1], 1))

  '''
    осуществляет обучение модели
  '''
  def fit(self,X,Y):
    X = self.addones(X)
    Y = np.reshape(Y, (len(Y), 1))
    W = self.init_weights(X)

    self.epoch_passed = 0
    self.learn_time = 0

    start = time.perf_counter()
    self.learn_func(X,Y,W)
    self.learn_time = time.perf_counter() - start


  '''
    возвращает отношение количества правильных ответов к общему
    количеству ответов обученной модели
  '''
  def score(self,X,Y):
    predictions = self.predict(X)
    scores = []
    for idx, y_pred in enumerate(predictions):
      # заполняем True если предсказание верное, иначе False
      scores.append(True if y_pred == Y[idx] else False)

    return scores.count(True)/len(scores)

  '''
    возвращает количество прошедших итераций обучения
  '''
  def n_iter(self):
    return self.epoch_passed

  '''
    возвращает время обучения
  '''
  def time(self):
    return self.learn_time    

  '''
    возвращает список вероятностей ответа на основании признаков
  '''
  def predict_proba(self,X):
    X = self.addones(X)

    return self.sigmoid_linear_regression(X,self.weights)

  '''
    предсказывает ответ обученной модели на основании признаков
  '''
  def predict(self,X):
    probas = self.predict_proba(X)
    for idx, y in enumerate(probas):
      probas[idx] = 1 if y > 0.5 else 0

    return probas

  '''
    добавляет единичный вектор к вектору признаков
  '''
  def addones(self,X):
    return np.hstack((np.ones((X.shape[0],1)),X))

  '''
    метод, реализующий стохастический градиентный спуск (SGD)
  '''
  def learn_sgd(self,X,Y,W):
    prev = self.cost_binary_cross_entropy(X,Y,W)
    for enum in range(self.epoch):
      self.epoch_passed = enum+1
      gradients = self.gradient(X,Y,W)
      W = W-self.learning_rate*gradients
      actual = self.cost_binary_cross_entropy(X,Y,W)
      diff = abs(prev - actual)
      if(diff <= self.stop_rate):
        break
      prev = actual
      
    self.weights = W
    return W

  '''
    метод, реализующий улучшенный градиентный спуск
    на основе rmsprop
  '''
  def learn_rmsprop(self,X,Y,W):
    prev = self.cost_binary_cross_entropy(X,Y,W)
    lr = self.learning_rate
    cached_rmsprop = [0] * len(W)
    decay_rate = 0.9
    for enum in range(self.epoch):
      self.epoch_passed = enum+1

      gradients = self.gradient(X,Y,W)
      NEW_W = []
      for i, (w, grad) in enumerate(zip(W, gradients)):
        cached_rmsprop[i] = decay_rate * cached_rmsprop[i] + (1-decay_rate) * grad **2
        new_w = w-lr*grad/(np.sqrt(cached_rmsprop[i]+1e-6))
        NEW_W.append(new_w)

      W = NEW_W
      actual = self.cost_binary_cross_entropy(X,Y,W)
      diff = abs(prev - actual)
      if(diff <= self.stop_rate):
        break
      prev = actual
      
    self.weights = W
    return W

  '''
    метод, реализующий улучшенный градиентный спуск
    на основе adam
  '''
  def learn_adam(self,X,Y,W):
    prev = self.cost_binary_cross_entropy(X,Y,W)
    lr = self.learning_rate
    m = [0] * len(W)
    v = [0] * len(W)
    t = 1
    beta1 = 0.9
    beta2 = 0.999
    for enum in range(self.epoch):
      self.epoch_passed = enum+1

      gradients = self.gradient(X,Y,W)
      NEW_W = []
      for i, (w, grad) in enumerate(zip(W, gradients)):
        m[i] = beta1 * m[i] + (1-beta1) * grad
        v[i] = beta2 * v[i] + (1-beta2) * grad **2
        m_corrected = m[i]/(1-beta1**t)
        v_corrected = v[i]/(1-beta2**t)

        new_w = w-lr*m_corrected/(np.sqrt(v_corrected+1e-8))

        NEW_W.append(new_w)

      t += 1

      W = NEW_W
      actual = self.cost_binary_cross_entropy(X,Y,W)
      diff = abs(prev - actual)
      if(diff <= self.stop_rate):
        break
      prev = actual
      
    self.weights = W
    return W

  '''
    метод, реализующий улучшенный градиентный спуск
    на основе adam
  '''
  def learn_nadam(self,X,Y,W):
    prev = self.cost_binary_cross_entropy(X,Y,W)
    lr = self.learning_rate
    m = [0] * len(W)
    v = [0] * len(W)
    t = 1
    beta1 = 0.9
    beta2 = 0.999
    for enum in range(self.epoch):
      self.epoch_passed = enum+1
      
      gradients = self.gradient(X,Y,W)
      NEW_W = []
      for i, (w, grad) in enumerate(zip(W, gradients)):
        m[i] = beta1 * m[i] + (1-beta1) * grad
        v[i] = beta2 * v[i] + (1-beta2) * grad **2
        m_corrected = m[i]/(1-beta1**t)
        v_corrected = v[i]/(1-beta2**t)

        new_w = w - (lr/(np.sqrt(v_corrected)+1e-8)*(beta1*m_corrected+(1-beta1)*grad/(1-beta1**t)))

        NEW_W.append(new_w)

      t += 1

      W = NEW_W
      actual = self.cost_binary_cross_entropy(X,Y,W)
      diff = abs(prev - actual)
      if(diff <= self.stop_rate):
        break
      prev = actual
      
    self.weights = W
    return W


  '''
    функция бинарной кросс-энтропии (векторная версия)
  '''
  def cost_binary_cross_entropy(self,X,Y,W):
    m = X.shape[0]
    total_cost = -(1 / m) * np.sum(
        Y * np.log(self.sigmoid_linear_regression(X, W)) + (1 - Y) * np.log(
            1 - self.sigmoid_linear_regression(X, W)))
    return total_cost    

  '''
    производная от функции бинарной кросс-энтропии (векторная версия)
  '''
  def gradient(self, X,Y,W):
    m = X.shape[0]
    return (1 / m) * np.dot(X.T, self.sigmoid_linear_regression(X,W) - Y)    

  '''
    главная функция логистической регрессии
  '''
  def sigmoid_linear_regression(self,X,W):
    return self.sigmoid(self.linear_regression(X,W))

  '''
    функция линейной регрессии
  '''
  def linear_regression(self,X,W):
    return np.dot(X,W)

  '''
    функция сигмоиды (логистическая функция)
  '''
  def sigmoid(self,H):
    return 1/(1+np.exp(-H))




## 3. Реализуйте метод градиентного спуска. Обучите логистическую регрессию этим методом. Выберете и посчитайте метрику качества. Метрика должна быть одинакова для всех пунктов домашнего задания. Для упрощения сравнения выберете только одну метрику.

В качестве метрики качества для разных алгоритмов градиентного спуска я выбрал три:
- точность результирующей модели (которая косвенно говорит о правильности нахождения минимума функции потерь)
- количество эпох для обучения, которые свидетельствуют о времени "схождения" функции потерь
- время обучения

In [4]:
results = pandas.DataFrame(columns=['Method','Score','Iterations','Time'])

Сначала проверим, какой результат дает реализация на базе SKLEARN

In [5]:
from sklearn.linear_model import LogisticRegression

start = time.perf_counter()

# SKLEARN с установками по-умолчанию
my_model = LogisticRegression()
my_model.fit(TRAIN_X,TRAIN_Y)
my_score = my_model.score(TEST_X,TEST_Y)
my_iter = my_model.n_iter_[0]
my_time = time.perf_counter() - start
print("SKLEARN DEFAULT score is: {}".format(my_score))
print("SKLEARN DEFAULT iter number: {}".format(my_iter))
print("SKLEARN DEFAULT time: {} sec".format(my_time))
results.loc[len(results)] = ['SKLEARN DEFAULT',my_score,my_iter,my_time]

SKLEARN DEFAULT score is: 0.9
SKLEARN DEFAULT iter number: 26
SKLEARN DEFAULT time: 0.018038465000003612 sec


Теперь попробуем собственную реализацию на основе SGD (Stochastic Gradient Descent)

In [6]:
# SGD (классический градиентный спуск)
my_model = MyLogisticRegression(learn_method='sgd',epoch=2000)
my_model.fit(TRAIN_X,TRAIN_Y)
my_score = my_model.score(TEST_X,TEST_Y)
my_iter = my_model.n_iter()
my_time = my_model.time()
print("SGD score is: {}".format(my_score))
print("SGD iter number: {}".format(my_iter))
print("SGD time: {} sec".format(my_time))
results.loc[len(results)] = ['MY SGD',my_score,my_iter,my_time]

SGD score is: 0.9
SGD iter number: 1561
SGD time: 0.11822678800000119 sec


## 4. Повторите п. 3 для метода скользящего среднего (Root Mean Square Propagation, RMSProp).


Пробуем реализацию на основе RMSProp

In [7]:
# RMSPROP
my_model = MyLogisticRegression(learn_method='rmsprop',epoch=2000)
my_model.fit(TRAIN_X,TRAIN_Y)
my_score = my_model.score(TEST_X,TEST_Y)
my_iter = my_model.n_iter()
my_time = my_model.time()
print("RMSPROP score is: {}".format(my_score))
print("RMSPROP iter number: {}".format(my_iter))
print("RMSPROP time: {} sec".format(my_time))
results.loc[len(results)] = ['MY RMSPROP',my_score,my_iter,my_time]

RMSPROP score is: 0.9
RMSPROP iter number: 733
RMSPROP time: 0.10216443600000957 sec


## 5. Повторите п. 3 для ускоренного по Нестерову метода адаптивной оценки моментов (Nesterov–accelerated Adaptive Moment Estimation, Nadam).

Пробуем реализацию на основе ADAM, а потом NADAM

In [8]:
# ADAM
my_model = MyLogisticRegression(learn_method='adam',epoch=2000)
my_model.fit(TRAIN_X,TRAIN_Y)
my_score = my_model.score(TEST_X,TEST_Y)
my_iter = my_model.n_iter()
my_time = my_model.time()
print("ADAM score is: {}".format(my_score))
print("ADAM iter number: {}".format(my_iter))
print("ADAM time: {} sec".format(my_time))
results.loc[len(results)] = ['MY ADAM',my_score,my_iter,my_time]

# NADAM
my_model = MyLogisticRegression(learn_method='nadam',epoch=2000)
my_model.fit(TRAIN_X,TRAIN_Y)
my_score = my_model.score(TEST_X,TEST_Y)
my_iter = my_model.n_iter()
my_time = my_model.time()
print("NADAM score is: {}".format(my_score))
print("NADAM iter number: {}".format(my_iter))
print("NADAM time: {} sec".format(my_time))
results.loc[len(results)] = ['MY NADAM',my_score,my_iter,my_time]

ADAM score is: 0.9
ADAM iter number: 800
ADAM time: 0.13908632499999385 sec
NADAM score is: 0.9
NADAM iter number: 797
NADAM time: 0.15446412599999348 sec


## 6. Сравните значение метрик для реализованных методов оптимизации. Можно оформить в виде таблицы вида |метод|метрика|время работы| (время работы опционально). Напишите вывод.


Метрики по результатам работы алгоритмов представлены ниже

In [9]:
results

,Method,Score,Iterations,Time
0,SKLEARN DEFAULT,0.9,26,0.018038
1,MY SGD,0.9,1561,0.118227
2,MY RMSPROP,0.9,733,0.102164
3,MY ADAM,0.9,800,0.139086
4,MY NADAM,0.9,797,0.154464
